In [1]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier

In [2]:

num_nominees = 9

np.random.choice(num_nominees, num_nominees, replace = False)

array([1, 8, 0, 2, 5, 7, 4, 3, 6])

In [3]:
?np.random.choice

In [4]:
full_table = pd.read_csv('./data/processed_results/osc_df')

# Setting Up our DataFrame

In [5]:
min_year = 1995

# # 80/20 Split (sequential) Excluding 2019
# year_range = 2019 - min_year
# train = full_table.loc[((full_table['year'] <= 2019 - 0.2 * year_range) & (full_table['year'] >= min_year))]
# valid = full_table.loc[((full_table['year'] > 2019 - 0.2 * year_range) & (full_table['year'] < 2019))]

# Training Set - Excluding 2019
train = full_table.loc[((full_table['year'] < 2019) & (full_table['year'] > min_year))]
test_2019 = full_table.loc[(full_table['year'] == 2019)]

print('training set contains:', train.shape[0], 'movies')
print('Prediciting on:', test_2019.shape[0], 'movies')

training set contains: 154 movies
Prediciting on: 9 movies


In [7]:
# Identify predictors and response

full_predictors = ['year','nom_gg_drama', 'winner_gg_drama', 'nom_gg_comedy', 'winner_gg_comedy',
       'nom_pga', 'winner_pga', 'nom_bafta', 'winner_bafta', 'nom_dga', 'winner_dga',
        'nom_sag', 'winner_sag', 'nom_cannes', 'winner_cannes','Nominations']

# Simulating a Voter using a Decision Tree

In [ ]:
voter1 = DecisionTreeClassifier(splitter='random',
                                max_depth=3,
                                min_samples_leaf=3,
                                random_state = 92)

In [39]:
def simulate_a_vote(model, train_df, to_predict_df, full_predictors):
    train = train_df.copy()
    test = to_predict_df.copy()
    
    # A noise column, randomly generated each time represents a voter's bias
    train.loc[:,'Noise'] = np.random.rand(train_df.shape[0])
    test.loc[:,'Noise'] = np.random.rand(to_predict_df.shape[0])

    # Looking at a random amount of awards shows (similar to bootstrapping) reflects a voter's attention to the season
    num_features = np.random.choice(len(full_predictors))
    voter_features = list(np.random.choice(full_predictors, num_features)) + ['Noise']

    x = np.array(train[voter_features])
    y = np.array(train['Oscar_win'])
    
    model.fit(x,y)
    # ProbA of the voter will represent the ranked votes
    ballot_clean = model.predict_proba(np.array(test[voter_features]))[:,1]

    # Add small random values to break up ties
    ballot = ballot_clean + np.random.rand(len(ballot_clean))/10000

    # Use np.argsort() to rank the order of the probA
    # The Academy uses ranked votes calculate winner
    rankings = np.abs(ballot.argsort() - len(ballot))
    return rankings

In [107]:
vote = simulate_a_vote(voter1, train, test_2019, full_predictors)
print("This voter's ballot looks like:",vote)
print("This means their first choice is:",list(test_2019.film)[np.argmin(vote)])

This voter's ballot looks like: [5 9 4 6 1 7 8 2 3]
This means their first choice is: Little Women (2019 film)


# Simulating the Entire Academy

In [41]:
def simulate_voting_body(num_voters, model, train_df, to_predict_df, full_predictors):
    collected_ballots = np.zeros((num_voters, to_predict_df.shape[0]))
    for i in range(num_voters):
        collected_ballots[i,:] = simulate_a_vote(model, train_df, to_predict_df, full_predictors)
    return collected_ballots

In [43]:
simulate_voting_body(5, voter1, train, test_2019, full_predictors)

array([[3., 1., 9., 6., 8., 2., 5., 7., 4.],
       [9., 4., 6., 3., 5., 8., 2., 7., 1.],
       [9., 5., 6., 4., 8., 2., 7., 3., 1.],
       [9., 7., 5., 4., 1., 2., 8., 6., 3.],
       [5., 4., 6., 9., 8., 7., 1., 2., 3.]])

In [133]:
def tally_votes(voting_body, list_of_nominees):
    # List of nominees must be in the same order as the vote index
    firsts = np.where(voting_body==1,1,0)
    tally = np.sum(firsts, axis = 0)
    tallied_votes_df = pd.DataFrame(tally, columns=['Votes']).T
    tallied_votes_df.columns = list_of_nominees
    return tallied_votes_df.T

In [134]:
this_academy = simulate_voting_body(1000, voter1, train, test_2019, full_predictors)
tally_votes(this_academy, list(test_2019.film)).sort_values('Votes', ascending = False)

,Votes
Once Upon a Time in Hollywood,264
Parasite (2019 film),238
Little Women (2019 film),118
1917 (2019 film),103
Marriage Story,95
The Irishman,54
Ford v Ferrari,52
Joker (2019 film),41
Jojo Rabbit,35


# Tiered Voting Changes

In [135]:
def remove_least(voting_body, list_of_nominees):
    # List of nominees must be in the same order as the vote index
    firsts = np.where(voting_body==1,1,0)
    tally = np.sum(firsts, axis = 0)
    least_votes_index = np.argmin(tally)
    # Removes the least voted entry (from # 1 to 0)
    voting_body = np.delete(voting_body, least_votes_index, axis = 1)
    list_of_nominees.remove(list_of_nominees[least_votes_index])
    return voting_body, list_of_nominees

In [ ]:
def redistribute_votes(voting_body):
    #### IN PROGRESS #####
    
    #every row goes from 1 to len(voting_body axis 1)
    for ballot in 
    rankings = np.abs(ballot.argsort() - len(ballot))
    
    return voting_body

In [136]:
tally_votes(this_academy, list(test_2019.film)).sort_values('Votes', ascending = False)

,Votes
Once Upon a Time in Hollywood,264
Parasite (2019 film),238
Little Women (2019 film),118
1917 (2019 film),103
Marriage Story,95
The Irishman,54
Ford v Ferrari,52
Joker (2019 film),41
Jojo Rabbit,35


In [137]:
new_votes, new_noms = remove_least(this_academy, list(test_2019.film))

print(len(new_noms))

tally_votes(new_votes, new_noms).sort_values('Votes', ascending = False)

8


,Votes
Once Upon a Time in Hollywood,264
Parasite (2019 film),238
Little Women (2019 film),118
1917 (2019 film),103
Marriage Story,95
The Irishman,54
Ford v Ferrari,52
Joker (2019 film),41


## Currently have to figure out how to re-rank every ballot